In [5]:
import os
print(os.getcwd())
print(os.listdir(os.getcwd()))
print(os.listdir(f'{os.getcwd()}/chromium/'))
print(f'{os.getcwd()}/chromium/chromium.chromiumdriver')

/home/burnett/projects/python/flask_server/flask_selenium_xvfb_bundle
['Dockerfile', 'Dockerfile:Zone.Identifier', 'chromium', 'app.py:Zone.Identifier', '.ipynb_checkpoints', 'requirements.txt', 'requirements.txt:Zone.Identifier', 'app.py']
['backup', 'chromium.chromiumdriver', 'chromium-browser']
/home/burnett/projects/python/flask_server/flask_selenium_xvfb_bundle/chromium/chromium.chromiumdriver


In [1]:
import tempfile
tmp_user_data_dir = tempfile.mkdtemp()
print(tmp_user_data_dir)

/tmp/tmpvsr_bi47


In [3]:
import os

chromedriver_path = os.path.join(
    os.getcwd(), "flask_selenium_xvfb_bundle", "chromium", "chromedriver.exe"
)

print("Driver path:", chromedriver_path)
print("Exists:", os.path.exists(chromedriver_path))
print("Is file:", os.path.isfile(chromedriver_path))

Driver path: /home/burnett/projects/python/flask_server/flask_selenium_xvfb_bundle/chromium/chromedriver.exe
Exists: True
Is file: True


In [3]:
import os
import uuid
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [10]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# TODO put chrome dependencies into the project directory (for portability)

# Set up Chrome options
options = Options()
# options.page_load_strategy = 'none'
# options.binary_location = "/usr/bin/chromium-browser"
# options.binary_location = f'{os.getcwd()}/chromium/chromium-browser'
options.binary_location = f'{os.getcwd()}/chromium/chrome-linux64/chrome'
print(options.binary_location)

# options.add_argument("--headless")  # Run without GUI
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Path to ChromeDriver binary (adjust if needed)
# chromedriver_path = os.path.expanduser("/snap/bin/chromium.chromedriver")
# chromedriver_path = f'{os.getcwd()}/chromium/chromium.chromiumdriver'
chromedriver_path = f'{os.getcwd()}/chromium/chromedriver'

print(chromedriver_path)


service = Service(executable_path=chromedriver_path)

# options.add_argument("--log-level=3")

# Use a temporary directory for Chrome user data
# profile_dir = f"/tmp/chrome-profile-{uuid.uuid4()}"
# profile_dir = f"{os.getcwd()}/tmp/chrome-profile-{uuid.uuid4()}"
# options.add_argument(f"--user-data-dir={profile_dir}")

# Create the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Use the driver
# driver.get("https://www.selenium.dev/selenium/web/web-form.html")
driver.get('https://www.seek.com.au/')
print("Page title:", driver.title)

# Clean up
driver.quit()

/home/burnett/projects/python/flask_server/flask_selenium_xvfb_bundle/chromium/chrome-linux64/chrome
/home/burnett/projects/python/flask_server/flask_selenium_xvfb_bundle/chromium/chromedriver
Page title: SEEK - Australia's no. 1 jobs, employment, career and recruitment site


In [17]:
from flask import Flask, request, jsonify
import subprocess
import time
import os
import tempfile
import uuid
import shutil

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

app = Flask(__name__)

def start_xvfb():
    """Starts virtual display if not already running."""
    if not os.environ.get("DISPLAY"):
        os.environ["DISPLAY"] = ":99"
        subprocess.Popen(["Xvfb", ":99", "-screen", "0", "1920x1080x24"])
        time.sleep(2)  # give Xvfb time to start

def get_data_with_selenium():
    """Selenium job to run with GUI in virtual display."""
    start_xvfb()

    # Set up Chrome options
    options = Options()
    options.page_load_strategy = 'none'
    options.binary_location = f'{os.getcwd()}/chromium/chrome-linux64/chrome'
    print(options.binary_location)
    options.add_argument("--disable-gpu")  # Optional
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    
    # Path to ChromeDriver binary (adjust if needed)
    chromedriver_path = f'{os.getcwd()}/chromium/chromedriver'
    print('chromedriver_path: ', chromedriver_path)
    
    service = Service(executable_path=chromedriver_path)

    # Use a temporary directory for Chrome user data
    profile_dir = f"{os.getcwd()}/tmp/chrome-profile-{uuid.uuid4()}"
    options.add_argument(f"--user-data-dir={profile_dir}")
    
    # Create the WebDriver
    driver = webdriver.Chrome(service=service, options=options)

    try:
        # Use the driver
        driver.get("https://www.selenium.dev/selenium/web/web-form.html")
        title = driver.title
        print("Page title:", title)
    finally:
        # Clean up
        driver.quit()
        time.sleep(2)
        shutil.rmtree(profile_dir, ignore_errors=True)


    return title
    
get_data_with_selenium()

/home/burnett/projects/python/flask_server/flask_selenium_xvfb_bundle/chromium/chrome-linux64/chrome
chromedriver_path:  /home/burnett/projects/python/flask_server/flask_selenium_xvfb_bundle/chromium/chromedriver
Page title: Web form


'Web form'